In [ ]:
import numpy as np
import pandas as pd
import ast

In [ ]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits= pd.read_csv("tmdb_5000_credits.csv")

In [ ]:
movies.head()

In [ ]:
# merging data frames using titles
movies = movies.merge(credits,on="title")

In [ ]:
movies.head(1)

In [ ]:
# Data coloum we gonna use
# genres
#keywords
#title
#overview
#cast
#crew

movies=movies[['title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
# iloc is used to select specific col or row
movies.iloc[0].genres

In [ ]:
# '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
# this into
# ['Action','Adventure','fantasy','Science Fiction']
#using ast literal to convert string to list
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [ ]:
#using apply() to apply function to entire data frame
movies['genres']=movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
movies.head()

In [ ]:
# on we only use first three cast of the movie for efficient recommendation
def convert3(obj):
    l=[]
    count=0
    for i in ast.literal_eval(obj):
        if count!=3:
            l.append(i['name'])
            count+=1
        else:
            break
    return l

In [ ]:
movies['cast']=movies['cast'].apply(convert3)

In [ ]:
# on we only use first director of the movie for efficient recommendation
def convert2(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']=="Director":
            l.append(i['name'])
    return l

In [ ]:
movies['crew']=movies['crew'].apply(convert2)

In [ ]:
movies.head()

In [ ]:
# overview column is in string format hence we will convert it into list using lamda function so we can easily concatenate required coloumn
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
#now to we will remove spaces between words so it will remain single entity
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
# now to merge and delete column as per requirement
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']
#lets create new data frame
new_df=movies[['title','tags']]

In [ ]:
new_df.head()

In [ ]:
#now lets convert tags list to string and lower case
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head()

In [ ]:
# Now we use steming for eliminating similar kind of words which are in different tenses like loved,loving,loves to love
#using nltk library
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

In [ ]:
# Vectorization of data in 2D Plane 
# creating vector of most repeating word except stop words
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
# Most repeatetive first 5000 words
cv.get_feature_names()

In [ ]:
# Now we calculate distance between vectors 
# we will calculate cosine distance instead of eucliden distance because of high amount of data
#(Cosine distance means angle btw vectors) using sklearn 
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
#Model
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distance=similarity[movie_index]
    movie_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movie_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Iron Man')